# Обучение YOLO 11 для детекции шахматных фигур с поддержкой Multi-Object Tracking

Этот notebook содержит полный процесс обучения модели YOLO 11 для детекции шахматных фигур с поддержкой трекинга объектов.

## Особенности:
- Использование YOLO 11 (ultralytics)
- Поддержка Multi-Object Tracking (MOT)
- 12 классов: белые и черные фигуры (pawn, rook, bishop, knight, king, queen)
- Возможность дообучения на пользовательских датасетах


In [ ]:
# Установка зависимостей
!pip install ultralytics
!pip install supervision
!pip install roboflow


In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import yaml
from typing import List, Tuple
import json


## Настройка путей и параметров


In [ ]:
# Настройки проекта
PROJECT_DIR = Path('/content/chess_pieces_yolo11')
DATASET_DIR = PROJECT_DIR / 'dataset'
MODELS_DIR = PROJECT_DIR / 'models'
RESULTS_DIR = PROJECT_DIR / 'results'

# Создание директорий
for dir_path in [PROJECT_DIR, DATASET_DIR, MODELS_DIR, RESULTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Параметры обучения
NUM_CLASSES = 12
CLASS_NAMES = [
    'white-pawn', 'white-rook', 'white-bishop', 'white-knight', 'white-king', 'white-queen',
    'black-pawn', 'black-rook', 'black-bishop', 'black-knight', 'black-king', 'black-queen'
]

# Параметры модели
MODEL_SIZE = 'n'  # n, s, m, l, x (nano, small, medium, large, xlarge)
IMG_SIZE = 640
EPOCHS = 100
BATCH_SIZE = 16
PATIENCE = 50  # Early stopping patience

print(f"Проект создан: {PROJECT_DIR}")
print(f"Количество классов: {NUM_CLASSES}")
print(f"Размер модели: {MODEL_SIZE}")


## Подготовка датасета

### Вариант 1: Использование существующего датасета (YOLO формат)
Если у вас уже есть датасет в формате YOLO (images + labels), укажите путь к нему.


In [ ]:
# Путь к существующему датасету (если есть)
EXISTING_DATASET_PATH = None  # Укажите путь, например: '/content/drive/MyDrive/chess_dataset'

def prepare_dataset_from_existing(path: str):
    """Подготовка датасета из существующей директории"""
    dataset_path = Path(path)
    
    # Проверка структуры
    train_images = dataset_path / 'train' / 'images'
    train_labels = dataset_path / 'train' / 'labels'
    val_images = dataset_path / 'valid' / 'images'
    val_labels = dataset_path / 'valid' / 'labels'
    
    if not all([train_images.exists(), train_labels.exists(), 
                val_images.exists(), val_labels.exists()]):
        raise ValueError("Датасет должен содержать train/images, train/labels, valid/images, valid/labels")
    
    # Создание символических ссылок или копирование
    target_train = DATASET_DIR / 'train'
    target_val = DATASET_DIR / 'valid'
    
    target_train.mkdir(exist_ok=True)
    target_val.mkdir(exist_ok=True)
    
    # Копирование данных
    import shutil
    shutil.copytree(train_images, target_train / 'images', dirs_exist_ok=True)
    shutil.copytree(train_labels, target_train / 'labels', dirs_exist_ok=True)
    shutil.copytree(val_images, target_val / 'images', dirs_exist_ok=True)
    shutil.copytree(val_labels, target_val / 'labels', dirs_exist_ok=True)
    
    return DATASET_DIR

if EXISTING_DATASET_PATH:
    dataset_path = prepare_dataset_from_existing(EXISTING_DATASET_PATH)
    print(f"Датасет подготовлен: {dataset_path}")
else:
    print("Используйте следующий блок для загрузки датасета из Roboflow или другого источника")


### Вариант 2: Загрузка датасета из Roboflow (если используется)


In [ ]:
# Загрузка датасета из Roboflow (раскомментируйте и укажите свои данные)
# from roboflow import Roboflow
# 
# rf = Roboflow(api_key="YOUR_API_KEY")
# project = rf.workspace("YOUR_WORKSPACE").project("YOUR_PROJECT")
# dataset = project.version(1).download("yolov11")
# 
# # Перемещение датасета
# import shutil
# shutil.move(dataset.location, DATASET_DIR)
# print(f"Датасет загружен: {DATASET_DIR}")


## Создание конфигурационного файла датасета


In [ ]:
def create_dataset_yaml(dataset_path: Path, class_names: List[str]) -> Path:
    """Создание YAML файла конфигурации датасета"""
    yaml_path = dataset_path / 'dataset.yaml'
    
    config = {
        'path': str(dataset_path.absolute()),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images' if (dataset_path / 'test' / 'images').exists() else None,
        'nc': len(class_names),
        'names': {i: name for i, name in enumerate(class_names)}
    }
    
    # Удаление None значений
    config = {k: v for k, v in config.items() if v is not None}
    
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
    print(f"Конфигурация датасета создана: {yaml_path}")
    print(f"Конфигурация:\n{yaml.dump(config, default_flow_style=False)}")
    
    return yaml_path

# Создание конфигурации (будет использовано после подготовки датасета)
# dataset_yaml = create_dataset_yaml(DATASET_DIR, CLASS_NAMES)


## Загрузка предобученной модели YOLO 11


In [ ]:
# Загрузка предобученной модели YOLO 11
model_name = f'yolo11{MODEL_SIZE}.pt'
model = YOLO(model_name)

print(f"Модель загружена: {model_name}")
print(f"Классы модели: {model.names}")


## Обучение модели

**Важно:** Перед обучением убедитесь, что:
1. Датасет подготовлен и находится в `DATASET_DIR`
2. Создан файл `dataset.yaml` с правильной конфигурацией
3. У вас достаточно GPU памяти (рекомендуется минимум 8GB для batch_size=16)


In [ ]:
# Обучение модели
# Убедитесь, что dataset_yaml указывает на правильный путь к конфигурации

def train_model(
    model: YOLO,
    dataset_yaml: str,
    epochs: int = EPOCHS,
    imgsz: int = IMG_SIZE,
    batch: int = BATCH_SIZE,
    patience: int = PATIENCE,
    device: str = '0',  # '0' для GPU, 'cpu' для CPU
    project: str = str(RESULTS_DIR),
    name: str = 'chess_pieces_yolo11',
    save: bool = True,
    save_period: int = 10,  # Сохранять чекпоинт каждые N эпох
):
    """Обучение модели YOLO 11"""
    
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        patience=patience,
        device=device,
        project=project,
        name=name,
        save=save,
        save_period=save_period,
        # Дополнительные параметры для улучшения качества
        optimizer='AdamW',  # AdamW обычно лучше для fine-tuning
        lr0=0.001,  # Начальная скорость обучения
        lrf=0.01,  # Финальная скорость обучения (lr0 * lrf)
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,  # Box loss gain
        cls=0.5,  # Class loss gain
        dfl=1.5,  # DFL loss gain
        pose=12.0,  # Pose loss gain (если используется)
        kobj=1.0,  # Keypoint obj loss gain (если используется)
        label_smoothing=0.0,
        nbs=64,  # Nominal batch size
        hsv_h=0.015,  # Image HSV-Hue augmentation
        hsv_s=0.7,  # Image HSV-Saturation augmentation
        hsv_v=0.4,  # Image HSV-Value augmentation
        degrees=0.0,  # Image rotation (+/- deg)
        translate=0.1,  # Image translation (+/- fraction)
        scale=0.5,  # Image scale (+/- gain)
        shear=0.0,  # Image shear (+/- deg)
        perspective=0.0,  # Image perspective (+/- fraction)
        flipud=0.0,  # Image flip up-down (probability)
        fliplr=0.5,  # Image flip left-right (probability)
        mosaic=1.0,  # Image mosaic (probability)
        mixup=0.0,  # Image mixup (probability)
        copy_paste=0.0,  # Segment copy-paste (probability)
    )
    
    return results

# Раскомментируйте для запуска обучения
# results = train_model(model, str(dataset_yaml), epochs=EPOCHS)


## Визуализация результатов обучения


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

def visualize_training_results(results_dir: Path, run_name: str = 'chess_pieces_yolo11'):
    """Визуализация результатов обучения"""
    results_path = results_dir / run_name
    
    # Поиск последнего запуска
    if not results_path.exists():
        # Поиск последней директории с результатами
        runs = sorted(results_path.parent.glob('*'), key=lambda x: x.stat().st_mtime, reverse=True)
        if runs:
            results_path = runs[0]
        else:
            print("Результаты обучения не найдены")
            return
    
    # Загрузка графиков
    results_image = results_path / 'results.png'
    confusion_matrix = results_path / 'confusion_matrix.png'
    train_batch = results_path / 'train_batch0.jpg'
    val_batch = results_path / 'val_batch0_pred.jpg'
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    if results_image.exists():
        img = plt.imread(results_image)
        axes[0, 0].imshow(img)
        axes[0, 0].axis('off')
        axes[0, 0].set_title('Training Results')
    
    if confusion_matrix.exists():
        img = plt.imread(confusion_matrix)
        axes[0, 1].imshow(img)
        axes[0, 1].axis('off')
        axes[0, 1].set_title('Confusion Matrix')
    
    if train_batch.exists():
        img = plt.imread(train_batch)
        axes[1, 0].imshow(img)
        axes[1, 0].axis('off')
        axes[1, 0].set_title('Train Batch')
    
    if val_batch.exists():
        img = plt.imread(val_batch)
        axes[1, 1].imshow(img)
        axes[1, 1].axis('off')
        axes[1, 1].set_title('Validation Predictions')
    
    plt.tight_layout()
    plt.show()
    
    # Вывод метрик
    metrics_file = results_path / 'results.csv'
    if metrics_file.exists():
        import pandas as pd
        df = pd.read_csv(metrics_file)
        print("\nПоследние метрики:")
        print(df.tail())
    
    return results_path

# Раскомментируйте после обучения
# best_model_path = visualize_training_results(RESULTS_DIR)


In [ ]:
def export_best_model(results_dir: Path, run_name: str = 'chess_pieces_yolo11', 
                      export_formats: List[str] = ['onnx', 'torchscript']):
    """Экспорт лучшей модели в различные форматы"""
    results_path = results_dir / run_name
    
    # Поиск последнего запуска
    if not results_path.exists():
        runs = sorted(results_path.parent.glob('*'), key=lambda x: x.stat().st_mtime, reverse=True)
        if runs:
            results_path = runs[0]
        else:
            print("Результаты обучения не найдены")
            return None
    
    best_model_path = results_path / 'weights' / 'best.pt'
    
    if not best_model_path.exists():
        print(f"Лучшая модель не найдена: {best_model_path}")
        return None
    
    # Загрузка лучшей модели
    best_model = YOLO(str(best_model_path))
    
    # Экспорт в различные форматы
    exported_models = {}
    for fmt in export_formats:
        try:
            export_path = best_model.export(format=fmt, imgsz=IMG_SIZE)
            exported_models[fmt] = export_path
            print(f"Модель экспортирована в {fmt}: {export_path}")
        except Exception as e:
            print(f"Ошибка при экспорте в {fmt}: {e}")
    
    # Копирование лучшей модели в директорию моделей
    import shutil
    target_model_path = MODELS_DIR / f'chess_pieces_yolo11_{MODEL_SIZE}_best.pt'
    shutil.copy(best_model_path, target_model_path)
    print(f"Лучшая модель скопирована: {target_model_path}")
    
    return {
        'best_pt': str(best_model_path),
        'exported': exported_models,
        'copied': str(target_model_path)
    }

# Раскомментируйте для экспорта модели
# export_info = export_best_model(RESULTS_DIR)


## Тестирование модели с Multi-Object Tracking

YOLO 11 поддерживает встроенный трекинг. Для использования трекинга нужно использовать метод `track()` вместо `predict()`.


In [ ]:
def test_model_with_tracking(
    model_path: str,
    video_path: str = None,
    image_path: str = None,
    conf_threshold: float = 0.25,
    iou_threshold: float = 0.45,
    tracker: str = 'bytetrack.yaml',  # 'bytetrack.yaml' или 'botsort.yaml'
    save: bool = True,
    show: bool = True,
):
    """Тестирование модели с поддержкой трекинга"""
    
    # Загрузка модели
    model = YOLO(model_path)
    
    if video_path:
        # Обработка видео с трекингом
        results = model.track(
            source=video_path,
            conf=conf_threshold,
            iou=iou_threshold,
            tracker=tracker,
            save=save,
            show=show,
            line_width=2,
        )
        
        # Результаты трекинга содержат track_ids
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes
                for i, box in enumerate(boxes):
                    # Получение track_id
                    track_id = int(box.id) if box.id is not None else None
                    cls = int(box.cls)
                    conf = float(box.conf)
                    
                    print(f"Track ID: {track_id}, Class: {model.names[cls]}, Confidence: {conf:.2f}")
    
    elif image_path:
        # Обработка изображения (трекинг не применим к одному изображению)
        results = model.predict(
            source=image_path,
            conf=conf_threshold,
            iou=iou_threshold,
            save=save,
            show=show,
            line_width=2,
        )
        
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes
                for i, box in enumerate(boxes):
                    cls = int(box.cls)
                    conf = float(box.conf)
                    print(f"Class: {model.names[cls]}, Confidence: {conf:.2f}")
    
    return results

# Пример использования (раскомментируйте и укажите пути)
# test_results = test_model_with_tracking(
#     model_path=str(MODELS_DIR / 'chess_pieces_yolo11_n_best.pt'),
#     video_path='/path/to/test_video.mp4',
#     conf_threshold=0.25,
#     tracker='bytetrack.yaml'
# )


## Сохранение конфигурации модели для использования в приложении


In [ ]:
def save_model_config(
    model_path: str,
    class_names: List[str],
    output_path: Path,
    model_size: str = 'n',
    img_size: int = 640,
    conf_threshold: float = 0.25,
    iou_threshold: float = 0.45,
):
    """Сохранение конфигурации модели для использования в приложении"""
    
    config = {
        'model_path': str(model_path),
        'model_size': model_size,
        'img_size': img_size,
        'num_classes': len(class_names),
        'class_names': class_names,
        'class_mapping': {i: name for i, name in enumerate(class_names)},
        'conf_threshold': conf_threshold,
        'iou_threshold': iou_threshold,
        'tracker': 'bytetrack.yaml',  # Используемый трекер
        'version': 'yolo11',
        'description': 'YOLO 11 модель для детекции шахматных фигур с поддержкой трекинга'
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    
    print(f"Конфигурация модели сохранена: {output_path}")
    return config

# Пример сохранения конфигурации
# model_config = save_model_config(
#     model_path=str(MODELS_DIR / 'chess_pieces_yolo11_n_best.pt'),
#     class_names=CLASS_NAMES,
#     output_path=MODELS_DIR / 'model_config.json',
#     model_size=MODEL_SIZE,
#     img_size=IMG_SIZE
# )
